### 构建多输入模型

In [2]:
from keras.models import Model
from keras import layers
from keras import Input

text_voca_size = 10000
question_voca_size = 10000
answer_voca_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_voca_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_voca_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 将编码后的问题和文本连接在一起
concatenated = layers.concatenate([encoded_text, encoded_question], axis=1)

# 最后添加一个softmax分类器
answer = layers.Dense(answer_voca_size, activation='softmax')(concatenated)

Metal device set to: Apple M1


In [3]:
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
             metrics=['acc'])

### 生成虚拟数据

In [13]:
import numpy as np
import keras

num_samples = 1000
max_len = 100
text = np.random.randint(1, text_voca_size, size=(num_samples, max_len))

In [14]:
text.shape

(1000, 100)

In [15]:
text[0]

array([6657, 2460, 7077, 9568, 3680, 9910, 2974, 6138, 5375, 7921, 7283,
       2067, 5807, 8711, 9730, 6762, 1221, 2258, 7663, 5553, 2236, 4218,
       6638, 7459, 5634, 4795, 6290, 5996, 1320, 4690, 9846, 6829, 6525,
       5242,  264, 3801, 6416, 2106, 3270, 8523, 9642, 5348, 3160, 5627,
       5469, 7359,  490, 9250, 6586, 4441, 6356, 6290, 1913, 2573, 1597,
       4766,   42, 9897, 3994, 9232, 1663, 1985, 3936,  452, 8294, 2885,
       8121, 1905, 9609, 4362, 5328, 9036, 8870, 5692,   70, 3698, 7818,
       8795, 8818,  704, 3630, 9636, 5214, 1361,  529, 2326, 5785, 1605,
       6056, 7764,  436, 8234,  245, 7949, 3545, 7744, 6608, 3426, 3305,
       3920])

In [16]:
question = np.random.randint(1, question_voca_size, size=(num_samples, max_len))

In [17]:
question.shape

(1000, 100)

In [18]:
answers = np.random.randint(answer_voca_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_voca_size)

In [19]:
history = model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10


2023-05-23 21:14:23.916560: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


8/8 [==============================] - 2s 102ms/step - loss: 6.2145 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 50ms/step - loss: 6.1947 - acc: 0.0230
Epoch 3/10
8/8 [==============================] - 0s 56ms/step - loss: 6.1333 - acc: 0.0020
Epoch 4/10
8/8 [==============================] - 0s 53ms/step - loss: 6.0600 - acc: 0.0070
Epoch 5/10
8/8 [==============================] - 0s 49ms/step - loss: 5.9984 - acc: 0.0090
Epoch 6/10
8/8 [==============================] - 0s 56ms/step - loss: 5.9394 - acc: 0.0120
Epoch 7/10
8/8 [==============================] - 0s 51ms/step - loss: 5.8367 - acc: 0.0160
Epoch 8/10
8/8 [==============================] - 0s 51ms/step - loss: 5.7446 - acc: 0.0250
Epoch 9/10
8/8 [==============================] - 0s 49ms/step - loss: 5.6483 - acc: 0.0320
Epoch 10/10
8/8 [==============================] - 0s 45ms/step - loss: 5.5678 - acc: 0.0380


### 构建多输出模型 

In [20]:
from keras import layers
from keras import Input
from keras.models import Model

voca_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, voca_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax',
                                name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])


In [21]:
model.compile(optimizer='rmsprop', 
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])